In [2]:
#NO REGULARISATION
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from sklearn.utils import shuffle
import pandas as pd 
from skimage.io import imread, imshow
from skimage.transform import rescale
from os import listdir
from sklearn.model_selection import train_test_split as tts
from skimage.color import rgb2hsv
from skimage.filters import threshold_otsu
from os import listdir
import pickle

In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
with open('refotsudataX','rb') as f:
    X = pickle.load(f)

with open('refotsudataY','rb') as g:
    Y = pickle.load(g)


In [5]:
X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.33)

In [6]:
X_train = X_train.reshape(6671,120,160,1)
X_train.shape
# y_train[0:20]

(6671, 120, 160, 1)

In [7]:
y_train_cate = keras.utils.to_categorical(
    y_train,num_classes=4,dtype='float32'
)

y_train_cate[0:10]

array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [9]:
model = keras.Sequential([
    #cnn layers
    keras.layers.Conv2D(filters=32,kernel_size=(3, 3),activation='relu',input_shape=(120,160,1)),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(filters=32,kernel_size=(3, 3),activation='relu'),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(filters=32,kernel_size=(3, 3),activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    
    keras.layers.Conv2D(filters=64,kernel_size=(3, 3),activation='relu'),
    keras.layers.MaxPooling2D((2,2)),

    #dense normal ann
    keras.layers.Flatten(), 
     
    keras.layers.Dense(90,activation='relu'),
    keras.layers.Dense(60,activation='relu'),
    keras.layers.Dense(30,activation='relu'),
    keras.layers.Dense(10,activation='relu'),
    
    keras.layers.Dense(4,activation='softmax'),
    #softmax normalises the probability of the output set
])

model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train,y_train_cate,epochs=30)

Epoch 1/30
209/209 [==============================] - 6s 26ms/step - loss: 1.3601 - accuracy: 0.3158
Epoch 2/30
209/209 [==============================] - 5s 26ms/step - loss: 1.0870 - accuracy: 0.5259
Epoch 3/30
209/209 [==============================] - 5s 26ms/step - loss: 0.8360 - accuracy: 0.6255
Epoch 4/30
209/209 [==============================] - 5s 26ms/step - loss: 0.7132 - accuracy: 0.6806
Epoch 5/30
209/209 [==============================] - 5s 26ms/step - loss: 0.6314 - accuracy: 0.7191
Epoch 6/30
209/209 [==============================] - 5s 26ms/step - loss: 0.5548 - accuracy: 0.7579
Epoch 7/30
209/209 [==============================] - 5s 26ms/step - loss: 0.4939 - accuracy: 0.7906
Epoch 8/30
209/209 [==============================] - 5s 26ms/step - loss: 0.4076 - accuracy: 0.8227
Epoch 9/30
209/209 [==============================] - 5s 26ms/step - loss: 0.3595 - accuracy: 0.8492
Epoch 10/30
209/209 [==============================] - 5s 26ms/step - loss: 0.3358 - accura

In [10]:
X_test = X_test.reshape(3286,120,160,1)
y_test_cate = keras.utils.to_categorical(
    y_test,num_classes=4,dtype='float32'
)

model.evaluate(X_test,y_test_cate)

103/103 [==============================] - 2s 17ms/step - loss: 2.0337 - accuracy: 0.7246


[2.033726453781128, 0.7245891690254211]

In [11]:
yprob = model.predict(X_test)
yprobcl = [np.argmax(i) for i in yprob]

In [12]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [13]:
accuracy_score(y_test,yprobcl)

0.7245891661594644

In [14]:
from sklearn.utils.multiclass import unique_labels
def plot(y_true,y_pred):    
    column = [f'Predicted {label}' for label in unique_labels(y_true)]
    indexx = [f'Actual {label}' for label in unique_labels(y_true)]
    table = pd.DataFrame(confusion_matrix(y_true,y_pred),columns=column,index=indexx)
    return table

plot(y_test,yprobcl)


,Predicted 0,Predicted 1,Predicted 2,Predicted 3
Actual 0,492,52,63,233
Actual 1,67,677,47,25
Actual 2,69,42,654,37
Actual 3,206,22,42,558
